<a href="https://www.kaggle.com/code/chickenugget/predicting-effectiveness-using-bow-rf-classifier?scriptVersionId=104397191" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd

df_train = pd.read_csv("../input/feedback-prize-effectiveness/train.csv",encoding = "ISO-8859-1")
df_test = pd.read_csv("../input/feedback-prize-effectiveness/test.csv",encoding = "ISO-8859-1")
df_submission = pd.read_csv("../input/feedback-prize-effectiveness/sample_submission.csv",encoding = "ISO-8859-1")

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV 
import string
string.punctuation

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import word_tokenize
from collections import Counter
from itertools import chain

# Remove stopwords 
stop = set(stopwords.words('english'))
df_train['discourse_text'].replace("[^a-zA-Z]"," ", regex=True, inplace =True)
df_train['discourse_text'] = df_train['discourse_text'].str.lower()
print(df_train)
df_train['discourse_text'] = df_train['discourse_text'].apply(lambda x: [item for item in str(x).split() if item not in stop])
freq = pd.Series(Counter(chain.from_iterable(df_train['discourse_text']))).sort_values(ascending=False)


       discourse_id      essay_id  \
0      0013cc385424  007ACE74B050   
1      9704a709b505  007ACE74B050   
2      c22adee811b6  007ACE74B050   
3      a10d361e54e4  007ACE74B050   
4      db3e453ec4e2  007ACE74B050   
...             ...           ...   
36760  9f63b687e76a  FFA381E58FC6   
36761  9d5bd7d86212  FFA381E58FC6   
36762  f1b78becd573  FFA381E58FC6   
36763  cc184624ca8e  FFA381E58FC6   
36764  c8a973681feb  FFA381E58FC6   

                                          discourse_text  \
0      hi  i m isaac  i m going to be writing about h...   
1      on my perspective  i think that the face is a ...   
2      i think that the face is a natural landform be...   
3      if life was on mars  we would know by now  the...   
4      people thought that the face was formed by ali...   
...                                                  ...   
36760  for many people they don t like only asking on...   
36761  also people have different views and opinions ...   
36762  advice i

In [3]:
!ln -s /kaggle/input/nltk-omw/omw-1.4 /usr/share/nltk_data/corpora/omw-1.4 

In [4]:
# Lemmatize all tokens into a new list: lemmatized
wordnet_lemmatizer = WordNetLemmatizer()
df_train['discourse_text'] = df_train['discourse_text'].apply(
                    lambda lst:[wordnet_lemmatizer.lemmatize(word, pos='v') for word in lst])
df_train['discourse_text'] = df_train['discourse_text'].apply(
                    lambda lst:[wordnet_lemmatizer.lemmatize(word, pos='n') for word in lst])
df_train['discourse_text'] = df_train['discourse_text'].apply(
                    lambda lst:[wordnet_lemmatizer.lemmatize(word, pos='r') for word in lst])
df_train['discourse_text'] = df_train['discourse_text'].apply(
                    lambda lst:[wordnet_lemmatizer.lemmatize(word, pos='a') for word in lst])
freq_final = pd.Series(Counter(chain.from_iterable(df_train['discourse_text']))).sort_values(ascending=False)


In [5]:
df_effects = pd.get_dummies(df_train.iloc[:,4])
df_train = pd.concat([df_train, df_effects], axis=1)

In [6]:
train = df_train
test = df_test


' '.join(str(x) for x in train.iloc[:,2])

headlines = []
for row in range(0,len(train.index)):
    headlines.append(' '.join(str(x) for x in train.iloc[row,2]))
headlines[0]



'hi isaac go write face mar natural landform life mar make story nasa take picture mar face see planet nasa know landform create life mar natural landform'

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier


## implement BAG OF WORDS
countvector=CountVectorizer(ngram_range=(1,2), token_pattern=r"(?u)\b\w+\b") # allows single-letter words in vocab.
traindataset=countvector.fit_transform(headlines)


In [8]:
# implement RandomForest Classifier
randomclassifier=RandomForestClassifier(n_estimators=100,criterion='entropy')
randomclassifier.fit(traindataset,train['discourse_effectiveness'])

# Predict on train dataset
train_predictions_prob = randomclassifier.predict_proba(traindataset)
train_predictions = randomclassifier.predict(traindataset)



In [9]:
# Confusion matrix
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
train_cm=confusion_matrix(train['discourse_effectiveness'],train_predictions)
print(train_cm)
train_score=accuracy_score(train['discourse_effectiveness'],train_predictions)
print(train_score)
train_report=classification_report(train['discourse_effectiveness'],train_predictions)
print(train_report)


[[20955     5    17]
 [   36  9287     3]
 [   51     1  6410]]
0.9969264245886033
              precision    recall  f1-score   support

    Adequate       1.00      1.00      1.00     20977
   Effective       1.00      1.00      1.00      9326
 Ineffective       1.00      0.99      0.99      6462

    accuracy                           1.00     36765
   macro avg       1.00      1.00      1.00     36765
weighted avg       1.00      1.00      1.00     36765



In [10]:
## Predict for the Test Dataset
test_transform= []
for row in range(0,len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row,2]))
test_dataset = countvector.transform(test_transform)
test_predictions_prob = randomclassifier.predict_proba(test_dataset)
test_predictions = randomclassifier.predict(test_dataset)
#test_predictions = randomclassifier.predict(test_dataset)

print(test_transform)
print(test_predictions)

["M a k i n g   c h o i c e s   i n   l i f e   c a n   b e   v e r y   d i f f i c u l t .   P e o p l e   o f t e n   a s k   f o r   a d v i c e   w h e n   t h e y   c a n   n o t   d e c i d e   o n   o n e   t h i n g .   I t ' s   a l w a y s   g o o d   t o   a s k   o t h e r s   f o r   t h e i r   a d v i c e   w h e n   m a k i n g   a   c h o i c e .   W h e n   y o u   h a v e   m u l t i p l e   o p i n i o n s   y o u   h a v e   t h e   a b i l i t y   t o   m a k e   t h e   b e s t   c h o i c e   f o r   y o u r s e l f .  ", 'S e e k i n g   m u l t i p l e   o p i n i o n s   c a n   h e l p   a   p e r s o n   m a k e   a   b e t t e r   c h o i c e  ', 'i t   c a n   d e c r e a s e   s t r e s s   l e v e l s  ', 'a   g r e a t   c h a n c e   t o   l e a r n   s o m e t h i n g   n e w  ', 'c a n   b e   v e r y   h e l p f u l   a n d   b e n e f i c i a l .  ', 'W h e n   m a k i n g   a   d e c i s i o n   t h e r e   i s   a   c h a n c e   t h a t   y o u

In [11]:
df_test_predict = pd.DataFrame(test_predictions_prob, columns=['Adequate','Effective','Ineffective'])
final_test_dataset = pd.merge(df_test, df_test_predict, how = 'right', left_index= True ,right_index =True)
final_test_dataset = final_test_dataset[['discourse_id', 'Ineffective','Adequate', 'Effective',]]



In [12]:
final_test_dataset.reset_index()
decimals = 2    
final_test_dataset.iloc[:,1:4] = final_test_dataset.iloc[:,1:4].apply(lambda x: round(x, decimals))
print(final_test_dataset)

   discourse_id  Ineffective  Adequate  Effective
0  a261b6e14276         0.51      0.40       0.09
1  5a88900e7dc1         0.51      0.40       0.09
2  9790d835736b         0.33      0.64       0.03
3  75ce6d68b67b         0.34      0.64       0.02
4  93578d946723         0.51      0.40       0.09
5  2e214524dbe3         0.51      0.40       0.09
6  84812fc2ab9f         0.49      0.43       0.08
7  c668ff840720         0.51      0.38       0.11
8  739a6d00f44a         0.51      0.40       0.09
9  bcfae2c9a244         0.51      0.40       0.09


In [13]:
df_submission.iloc[:,1] = final_test_dataset.iloc[:,1]
df_submission.iloc[:,2] = final_test_dataset.iloc[:,2]
df_submission.iloc[:,3] = final_test_dataset.iloc[:,3]
print(df_submission)
df_submission.to_csv('submission.csv', index=False)

   discourse_id  Ineffective  Adequate  Effective
0  a261b6e14276         0.51      0.40       0.09
1  5a88900e7dc1         0.51      0.40       0.09
2  9790d835736b         0.33      0.64       0.03
3  75ce6d68b67b         0.34      0.64       0.02
4  93578d946723         0.51      0.40       0.09
5  2e214524dbe3         0.51      0.40       0.09
6  84812fc2ab9f         0.49      0.43       0.08
7  c668ff840720         0.51      0.38       0.11
8  739a6d00f44a         0.51      0.40       0.09
9  bcfae2c9a244         0.51      0.40       0.09
